In [1]:
from zander_preparation_parametrized import *
from qrisp.quantum_backtracking import OHQInt
from qrisp import QuantumArray
from classical_cost_func import cost_function, format_coeffs, new_cost_function, cost_symp
import numpy as np
import matplotlib.pyplot as plt
from phase_polynomial import *
from mixer_workaround import * 
from qrisp.qaoa import *


from encode_BMW_problem import PBS_graph, N, M, cost_coeff

/Users/matteoantonioinajetovic/AirbusBMWChallenge/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
tot_coeff = format_coeffs(cost_coeff, N )

qtype = OHQInt(N)
q_array = QuantumArray(qtype = qtype, shape = (M))

#cost op
P,S = cost_symp(tot_coeff,M,N,PBS_graph)
ord_symbs=list(S.values())

def cost_op(q_array, gamma):
    app_sb_phase_polynomial(q_array, -gamma*P, ord_symbs)

#init func
init_func = pbs_state_init(PBS_graph, 0 , N)

#mixer
mixer_op = mixer(PBS_graph,0,N)

#classical cost function
values = []
cl_cost = new_cost_function(tot_coeff,M,N,PBS_graph,values)

# run the QAOA
qaoaPBS = QAOAProblem(cost_operator=cost_op ,mixer=mixer_op, cl_cost_function=cl_cost)
qaoaPBS.set_init_function(init_func)
depth = 2#int(np.log2(M*N))

In [3]:
circ,symbols=qaoaPBS.compile_circuit(q_array,depth)

In [4]:
def evaluate_cost(theta,symbols):
    subs_dic = {symbols[i] : theta[i] for i in range(len(symbols))}
    res_dic = q_array.get_measurement(subs_dic = subs_dic, precompiled_qc = circ)
    cost = cl_cost(res_dic)
    return cost

In [5]:
def finite_diff(theta,index,gb,eps=.2):
    #inspired by qiskit  
    e_i = np.identity(theta.size)[:, index]
    plus = theta + eps * e_i
    minus = theta - eps * e_i
    grad = (evaluate_cost(plus,gb) - evaluate_cost(minus,gb)) / (2 * eps)
    return grad

In [6]:

def compute_gradients(gb, grad_func, num_samples=10):
    grads = []
    index = len(gb)-1 # wrt last parameter

    for _ in range(num_samples): #sampling from random samples
        theta = 2*np.pi* np.random.rand(len(gb)) 
        grad = grad_func(theta,index,gb)
        grads.append(grad)
    grads = np.array(grads)
    return grads

In [ ]:
gradients=compute_gradients(symbols,finite_diff,100)

In [8]:
np.var(gradients)

0.14037471189217812